In [ ]:
def imprime_solucao(solucao_dict, titulo, salas_dict):
    """
    Imprime uma solução em formato legível
    
    Args:
        solucao_dict: {Disciplina: (Sala.id, horario)}
        titulo: string do título
        salas_dict: dicionário {Sala.id: Sala}
    """
    print(f"\n{'='*70}")
    print(f"{titulo}")
    print(f"{'='*70}")
    
    if solucao_dict is None:
        print("  Nenhuma solução encontrada!")
        return
    
    # Ordenar por horário para melhor visualização
    solucao_ordenada = sorted(solucao_dict.items(), key=lambda x: x[1][1])
    
    print(f"\n{'Disciplina':<25} {'Sala':<10} {'Horário':<15} {'Professor':<15} {'Desperdício':<12}")
    print("-" * 77)
    
    custo_total = 0
    for disciplina, (sala_id, horario) in solucao_ordenada:
        sala = salas_dict[sala_id]
        desperdicio = calcula_desperdicio(disciplina, sala)
        custo_total += desperdicio
        
        print(f"{disciplina.id:<25} {sala_id:<10} {horario:<15} {disciplina.professor:<15} {desperdicio:<12}")
    
    print("-" * 77)
    print(f"{'TOTAL DE DESPERDÍCIO':<25} {'':<10} {'':<15} {'':<15} {custo_total:<12}")
    print(f"{'='*70}\n")


# Imprimir soluções
if resultado_a_estrela['encontrou']:
    imprime_solucao(resultado_a_estrela['solucao'], "✓ SOLUÇÃO DO A*", salas_dict)

if resultado_iddfs['encontrou']:
    imprime_solucao(resultado_iddfs['solucao'], "✓ SOLUÇÃO DO IDDFS", salas_dict)

# Verificações finais
print("\n✓ Verificação de Restrições (A*):")
if resultado_a_estrela['encontrou']:
    solucao = resultado_a_estrela['solucao']
    print(f"  • Total de disciplinas alocadas: {len(solucao)}/{len(disciplinas)}")
    
    profs_por_horario = defaultdict(set)
    salas_por_horario = defaultdict(set)
    
    for disciplina, (sala_id, horario) in solucao.items():
        profs_por_horario[horario].add(disciplina.professor)
        salas_por_horario[horario].add(sala_id)
    
    conflito_prof = False
    for horario, profs in profs_por_horario.items():
        if len(profs) != len([d for d, (_, h) in solucao.items() if h == horario]):
            conflito_prof = True
            print(f"  ✗ CONFLITO DE PROFESSOR NO {horario}!")
    
    if not conflito_prof:
        print(f"  ✓ Nenhum conflito de professor")
    
    conflito_cap = False
    for disciplina, (sala_id, _) in solucao.items():
        sala = salas_dict[sala_id]
        if sala.capacidade < disciplina.num_alunos:
            conflito_cap = True
            print(f"  ✗ CAPACIDADE INSUFICIENTE: {disciplina.id} em {sala_id}!")
    
    if not conflito_cap:
        print(f"  ✓ Todas as capacidades respeitadas")

## 10. Impressão das Soluções

Formato legível para cada algoritmo

In [ ]:
# Preparar dados para comparação
dados_comparacao = []

if resultado_a_estrela['encontrou']:
    dados_comparacao.append({
        'Algoritmo': 'A*',
        'Custo (Desperdício)': resultado_a_estrela['custo'],
        'Nós Gerados': resultado_a_estrela['nos_gerados'],
        'Tempo (s)': f"{resultado_a_estrela['tempo']:.4f}"
    })

if resultado_iddfs['encontrou']:
    dados_comparacao.append({
        'Algoritmo': 'IDDFS',
        'Custo (Desperdício)': resultado_iddfs['custo'],
        'Nós Gerados': resultado_iddfs['nos_gerados'],
        'Tempo (s)': f"{resultado_iddfs['tempo']:.4f}"
    })

# Criar DataFrame
df_comparacao = pd.DataFrame(dados_comparacao)

print("\n📊 TABELA COMPARATIVA")
print("=" * 70)
print(df_comparacao.to_string(index=False))
print("=" * 70)

# Análise
print("\n🔬 ANÁLISE:")
if resultado_a_estrela['encontrou'] and resultado_iddfs['encontrou']:
    print(f"\n• Ambos encontraram solução com MESMO custo: {resultado_a_estrela['custo']}")
    print(f"  → A* gerou {resultado_a_estrela['nos_gerados']} nós")
    print(f"  → IDDFS gerou {resultado_iddfs['nos_gerados']} nós")
    
    if resultado_a_estrela['nos_gerados'] < resultado_iddfs['nos_gerados']:
        reducao = (1 - resultado_a_estrela['nos_gerados'] / resultado_iddfs['nos_gerados']) * 100
        print(f"  → A* gerou {reducao:.1f}% menos nós (mais eficiente com heurística)")
    else:
        aumento = (resultado_iddfs['nos_gerados'] / resultado_a_estrela['nos_gerados'] - 1) * 100
        print(f"  → IDDFS gerou {aumento:.1f}% mais nós (custo de iterar profundidades)")
    
    if resultado_a_estrela['tempo'] < resultado_iddfs['tempo']:
        reducao = (1 - resultado_a_estrela['tempo'] / resultado_iddfs['tempo']) * 100
        print(f"  → A* foi {reducao:.1f}% mais rápido")
    else:
        aumento = (resultado_iddfs['tempo'] / resultado_a_estrela['tempo'] - 1) * 100
        print(f"  → IDDFS foi {aumento:.1f}% mais lento")

## 9. Comparação Experimental

Tabela comparativa dos dois algoritmos

In [ ]:
print("🔍 Executando A* ...\n")
resultado_a_estrela = busca_a_estrela(salas, disciplinas, horarios, salas_dict)

if resultado_a_estrela['encontrou']:
    print(f"✓ A* encontrou solução!")
    print(f"  Custo (desperdício total): {resultado_a_estrela['custo']}")
    print(f"  Nós gerados: {resultado_a_estrela['nos_gerados']}")
    print(f"  Tempo: {resultado_a_estrela['tempo']:.4f}s\n")
else:
    print("✗ A* NÃO encontrou solução\n")

print("-" * 70)
print("\n🔍 Executando IDDFS ...\n")
resultado_iddfs = busca_iddfs(salas, disciplinas, horarios, salas_dict, max_profundidade=20)

if resultado_iddfs['encontrou']:
    print(f"✓ IDDFS encontrou solução!")
    print(f"  Custo (desperdício total): {resultado_iddfs['custo']}")
    print(f"  Nós gerados: {resultado_iddfs['nos_gerados']}")
    print(f"  Tempo: {resultado_iddfs['tempo']:.4f}s\n")
else:
    print("✗ IDDFS NÃO encontrou solução\n")

print("=" * 70)

## 8. Execução dos Algoritmos

In [ ]:
# ============================================================================
# INSTÂNCIA DE EXEMPLO OBRIGATÓRIA
# ============================================================================

# Salas
sala_a = Sala('Sala A', 30)
sala_b = Sala('Sala B', 50)
sala_c = Sala('Sala C', 100)
salas = [sala_a, sala_b, sala_c]
salas_dict = {s.id: s for s in salas}

# Disciplinas
calc_i = Disciplina('Cálculo I', 'Prof. Ana', 45)
algebra = Disciplina('Álgebra Linear', 'Prof. Ana', 25)
prog_i = Disciplina('Programação I', 'Prof. Bruno', 30)
estrut_dados = Disciplina('Estruturas de Dados', 'Prof. Carlos', 40)
disciplinas = [calc_i, algebra, prog_i, estrut_dados]

# Horários
horarios = ['Seg 08-10', 'Seg 10-12', 'Ter 08-10']

# Exibir instância
print("=" * 70)
print("INSTÂNCIA DE EXEMPLO")
print("=" * 70)
print("\n📚 SALAS:")
for sala in salas:
    print(f"  • {sala.id}: capacidade = {sala.capacidade}")

print("\n👥 DISCIPLINAS:")
for disc in disciplinas:
    print(f"  • {disc.id}: Prof. {disc.professor}, {disc.num_alunos} alunos")

print("\n⏰ HORÁRIOS:")
for h in horarios:
    print(f"  • {h}")

print("\n" + "=" * 70)

## 7. Instância de Exemplo Obrigatória

Conforme especificação do Tema 6 - UFPB IA

In [ ]:
# ============================================================================
# ALGORITMO IDDFS (Iterative Deepening DFS)
# ============================================================================

def dfs_limitada(estado, limite, salas, disciplinas, horarios, salas_dict, nos_info):
    """
    DFS com limite de profundidade
    
    Args:
        estado: Objeto Estado atual
        limite: profundidade máxima
        salas, disciplinas, horarios, salas_dict: dados do problema
        nos_info: dict com contagem de nós {'nos': int}
        
    Returns:
        dict: {
            'encontrou': bool,
            'solucao': {Disciplina: (Sala.id, horario)} ou None,
            'custo': int ou None
        }
    """
    nos_info['nos'] += 1
    
    # Se atingiu objetivo
    if testa_objetivo(estado, disciplinas):
        custo = calcula_custo_estado(estado, salas_dict, disciplinas)
        return {
            'encontrou': True,
            'solucao': estado.alocacoes,
            'custo': custo
        }
    
    # Se atingiu limite, para
    if limite == 0:
        return {'encontrou': False, 'solucao': None, 'custo': None}
    
    # Gera sucessores e explora
    sucessores = gera_sucessores(estado, salas, disciplinas, horarios, salas_dict)
    
    for novo_estado, _, _, _ in sucessores:
        resultado = dfs_limitada(novo_estado, limite - 1, salas, disciplinas, horarios, salas_dict, nos_info)
        if resultado['encontrou']:
            return resultado
    
    return {'encontrou': False, 'solucao': None, 'custo': None}


def busca_iddfs(salas, disciplinas, horarios, salas_dict, max_profundidade=20):
    """
    IDDFS: Iterative Deepening DFS
    
    Aumenta o limite de profundidade de 1 até max_profundidade, executando DFS
    limitada a cada iteração.
    
    Args:
        salas: lista de Sala
        disciplinas: lista de Disciplina
        horarios: lista de strings de horário
        salas_dict: dicionário {Sala.id: Sala}
        max_profundidade: limite máximo de profundidade
        
    Returns:
        dict: {
            'solucao': {Disciplina: (Sala.id, horario)},
            'custo': int,
            'nos_gerados': int,
            'tempo': float,
            'encontrou': bool
        }
    """
    tempo_inicio = time.time()
    nos_gerados = 0
    estado_inicial = Estado()
    
    for profundidade in range(1, max_profundidade + 1):
        nos_info = {'nos': 0}
        resultado = dfs_limitada(estado_inicial, profundidade, salas, disciplinas, horarios, salas_dict, nos_info)
        nos_gerados += nos_info['nos']
        
        if resultado['encontrou']:
            tempo_total = time.time() - tempo_inicio
            return {
                'solucao': resultado['solucao'],
                'custo': resultado['custo'],
                'nos_gerados': nos_gerados,
                'tempo': tempo_total,
                'encontrou': True
            }
    
    tempo_total = time.time() - tempo_inicio
    return {
        'solucao': None,
        'custo': None,
        'nos_gerados': nos_gerados,
        'tempo': tempo_total,
        'encontrou': False
    }


print("Algoritmo IDDFS definido!")

## 6. Algoritmo IDDFS (Iterative Deepening DFS)

Implementação híbrida BFS/DFS que:
1. Aumenta o limite de profundidade gradualmente
2. Executa DFS limitada em cada iteração
3. Retorna primeira solução encontrada (completo e ótimo)

In [ ]:
# ============================================================================
# ALGORITMO A*
# ============================================================================

def busca_a_estrela(salas, disciplinas, horarios, salas_dict):
    """
    Implementação de A* para resolver o problema de timetabling
    
    Args:
        salas: lista de Sala
        disciplinas: lista de Disciplina
        horarios: lista de strings de horário
        salas_dict: dicionário {Sala.id: Sala}
        
    Returns:
        dict: {
            'solucao': {Disciplina: (Sala.id, horario)},
            'custo': int,
            'nos_gerados': int,
            'tempo': float,
            'encontrou': bool
        }
    """
    tempo_inicio = time.time()
    nos_gerados = 0
    visitados = set()
    
    # Estado inicial
    estado_inicial = Estado()
    g_inicial = 0
    h_inicial = heuristica_admissivel(estado_inicial, salas, disciplinas, salas_dict)
    f_inicial = g_inicial + h_inicial
    
    # Fila de prioridade: (f(n), contador, estado, g(n))
    # contador: para quebrar empates de forma determinística
    contador = 0
    fila = [(f_inicial, contador, estado_inicial, g_inicial)]
    contador += 1
    
    while fila:
        f_n, _, estado_atual, g_n = heapq.heappop(fila)
        
        # Verifica se já visitado
        estado_key = tuple(sorted((d.id, s, h) for d, (s, h) in estado_atual.alocacoes.items()))
        if estado_key in visitados:
            continue
        visitados.add(estado_key)
        
        # Testa objetivo
        if testa_objetivo(estado_atual, disciplinas):
            tempo_total = time.time() - tempo_inicio
            return {
                'solucao': estado_atual.alocacoes,
                'custo': g_n,
                'nos_gerados': nos_gerados,
                'tempo': tempo_total,
                'encontrou': True
            }
        
        # Gera sucessores
        sucessores = gera_sucessores(estado_atual, salas, disciplinas, horarios, salas_dict)
        
        for novo_estado, disciplina, sala, horario in sucessores:
            nos_gerados += 1
            
            # Calcula novo g(n+1)
            g_novo = calcula_custo_estado(novo_estado, salas_dict, disciplinas)
            
            # Calcula h(n+1)
            h_novo = heuristica_admissivel(novo_estado, salas, disciplinas, salas_dict)
            f_novo = g_novo + h_novo
            
            novo_estado_key = tuple(sorted((d.id, s, h) for d, (s, h) in novo_estado.alocacoes.items()))
            
            if novo_estado_key not in visitados:
                heapq.heappush(fila, (f_novo, contador, novo_estado, g_novo))
                contador += 1
    
    # Sem solução encontrada
    tempo_total = time.time() - tempo_inicio
    return {
        'solucao': None,
        'custo': None,
        'nos_gerados': nos_gerados,
        'tempo': tempo_total,
        'encontrou': False
    }


print("Algoritmo A* definido!")

## 5. Algoritmo A*

Implementação de A* com:
- **g(n)**: custo acumulado (desperdício total até agora)
- **h(n)**: heurística admissível (estimativa do desperdício futuro)
- **f(n)**: g(n) + h(n) (custo estimado total)

Usa fila de prioridade (heapq) ordenada por f(n).

In [ ]:
# ============================================================================
# HEURÍSTICA ADMISSÍVEL: h(n)
# ============================================================================

def heuristica_admissivel(estado, salas, disciplinas, salas_dict):
    """
    Heurística admissível para A*:
    h(n) = soma do menor desperdício possível para cada disciplina não alocada
    
    Para cada disciplina não alocada, considera o desperdício mínimo possível
    escolhendo a sala com menor desperdício que a comporta.
    
    Args:
        estado: Objeto Estado atual
        salas: lista de Sala
        disciplinas: lista de Disciplina
        salas_dict: dicionário {Sala.id: Sala}
        
    Returns:
        int: valor da heurística
    """
    nao_alocadas = obtem_disciplinas_nao_alocadas(estado, disciplinas)
    
    h_valor = 0
    
    for disciplina in nao_alocadas:
        # Encontra o menor desperdício possível para esta disciplina
        menor_desperdicio = float('inf')
        
        for sala in salas:
            if verifica_capacidade(disciplina, sala):
                desperdicio = calcula_desperdicio(disciplina, sala)
                menor_desperdicio = min(menor_desperdicio, desperdicio)
        
        # Se nenhuma sala comporta, retorna infinito (estado inviável)
        if menor_desperdicio == float('inf'):
            return float('inf')
        
        h_valor += menor_desperdicio
    
    return h_valor


print("Heurística admissível definida!")

## 4. Heurística Admissível para A*

A heurística h(n) é a **soma do menor desperdício possível para cada disciplina não alocada**, considerando apenas as salas que a comportam.

Esta heurística é **admissível** porque:
- Nunca sobreestima (cada disciplina usa pelo menos a sala com menor desperdício que a cabe)
- É consistente (nunca diminui ao longo de um caminho)

In [ ]:
# ============================================================================
# FUNÇÕES AUXILIARES PARA BUSCA
# ============================================================================

def obtem_disciplinas_nao_alocadas(estado, disciplinas):
    """
    Retorna lista de disciplinas ainda não alocadas
    
    Args:
        estado: Objeto Estado
        disciplinas: lista de Disciplina
        
    Returns:
        list: disciplinas não presentes em estado.alocacoes
    """
    return [d for d in disciplinas if d not in estado.alocacoes]


def testa_objetivo(estado, disciplinas):
    """
    Testa se todas as disciplinas foram alocadas (estado objetivo)
    
    Args:
        estado: Objeto Estado
        disciplinas: lista de Disciplina
        
    Returns:
        bool: True se todas alocadas
    """
    return len(estado.alocacoes) == len(disciplinas)


def gera_sucessores(estado, salas, disciplinas, horarios, salas_dict):
    """
    Gera todos os sucessores válidos a partir de um estado
    
    Estratégia: escolhe primeira disciplina não alocada e tenta todos (sala, horário)
    
    Args:
        estado: Objeto Estado atual
        salas: lista de Sala
        disciplinas: lista de Disciplina
        horarios: lista de strings de horário
        salas_dict: dicionário {Sala.id: Sala}
        
    Returns:
        list: [(novo_estado, disciplina_alocada, sala, horario), ...]
    """
    nao_alocadas = obtem_disciplinas_nao_alocadas(estado, disciplinas)
    
    if not nao_alocadas:
        return []  # Nenhum sucessor se todas alocadas
    
    # Seleciona a primeira não alocada (estratégia simples)
    disciplina = nao_alocadas[0]
    
    sucessores = []
    
    for sala in salas:
        for horario in horarios:
            if pode_alocar(disciplina, sala, horario, estado):
                # Criar novo estado com esta alocação
                novo_estado = estado.copiar()
                novo_estado.alocacoes[disciplina] = (sala.id, horario)
                
                # Atualizar professores_ocupados
                if horario not in novo_estado.professores_ocupados:
                    novo_estado.professores_ocupados[horario] = set()
                novo_estado.professores_ocupados[horario].add(disciplina.professor)
                
                sucessores.append((novo_estado, disciplina, sala, horario))
    
    return sucessores


print("Funções auxiliares definidas!")

## 3. Funções Auxiliares para Busca

In [ ]:
# ============================================================================
# FUNÇÕES DE VALIDAÇÃO E RESTRIÇÕES
# ============================================================================

def verifica_capacidade(disciplina, sala):
    """
    Restrição 1: Verifica se a sala tem capacidade suficiente para a disciplina
    
    Args:
        disciplina: Objeto Disciplina
        sala: Objeto Sala
        
    Returns:
        bool: True se capacidade >= num_alunos
    """
    return sala.capacidade >= disciplina.num_alunos


def verifica_conflito_professor(disciplina, horario, estado):
    """
    Restrição 2: Verifica se o professor já está ocupado naquele horário
    
    Args:
        disciplina: Objeto Disciplina
        horario: string do horário (ex: 'Seg 08-10')
        estado: Objeto Estado com professores_ocupados
        
    Returns:
        bool: True se sem conflito (professor livre), False se conflito
    """
    if horario not in estado.professores_ocupados:
        return True
    return disciplina.professor not in estado.professores_ocupados[horario]


def pode_alocar(disciplina, sala, horario, estado):
    """
    Verifica se é possível alocar uma disciplina em uma sala e horário
    
    Args:
        disciplina: Objeto Disciplina
        sala: Objeto Sala
        horario: string do horário
        estado: Objeto Estado atual
        
    Returns:
        bool: True se ambas as restrições são satisfeitas
    """
    return (verifica_capacidade(disciplina, sala) and 
            verifica_conflito_professor(disciplina, horario, estado))


def calcula_desperdicio(disciplina, sala):
    """
    Calcula o desperdício de capacidade para uma alocação
    
    desperdício = capacidade_sala - num_alunos_disciplina
    
    Args:
        disciplina: Objeto Disciplina
        sala: Objeto Sala
        
    Returns:
        int: valor do desperdício
    """
    return sala.capacidade - disciplina.num_alunos


def calcula_custo_estado(estado, salas_dict, disciplinas):
    """
    Calcula o custo total (g(n)) = soma dos desperdícios de todas alocações
    
    Args:
        estado: Objeto Estado
        salas_dict: dicionário {Sala.id: Sala}
        disciplinas: lista de Disciplina
        
    Returns:
        int: custo total do estado
    """
    custo = 0
    for disciplina, (sala_id, _) in estado.alocacoes.items():
        sala = salas_dict[sala_id]
        custo += calcula_desperdicio(disciplina, sala)
    return custo


print("Funções de validação e restrições definidas!")

## 2. Funções de Validação e Restrições

In [ ]:
# ============================================================================
# ESTRUTURAS DE DADOS PARA O PROBLEMA
# ============================================================================

class Sala:
    """Representa uma sala física com capacidade fixa"""
    def __init__(self, id, capacidade):
        self.id = id  # identificador (ex: 'Sala A')
        self.capacidade = capacidade  # número máximo de alunos

    def __repr__(self):
        return f"Sala({self.id}, cap={self.capacidade})"

    def __eq__(self, other):
        return isinstance(other, Sala) and self.id == other.id

    def __hash__(self):
        return hash(self.id)


class Disciplina:
    """Representa uma disciplina com professor e número de alunos"""
    def __init__(self, id, professor, num_alunos):
        self.id = id  # identificador único
        self.professor = professor  # nome do professor
        self.num_alunos = num_alunos  # número de alunos inscritos

    def __repr__(self):
        return f"Disciplina({self.id}, prof={self.professor}, n={self.num_alunos})"

    def __eq__(self, other):
        return isinstance(other, Disciplina) and self.id == other.id

    def __hash__(self):
        return hash(self.id)


class Estado:
    """Representa um estado parcial de alocação"""
    def __init__(self, alocacoes=None, professores_ocupados=None):
        # alocacoes: {Disciplina: (Sala, horario)}
        self.alocacoes = alocacoes if alocacoes is not None else {}
        # professores_ocupados: {horario: set(Professores)}
        self.professores_ocupados = professores_ocupados if professores_ocupados is not None else {}

    def copiar(self):
        """Cria uma cópia profunda do estado"""
        return Estado(
            deepcopy(self.alocacoes),
            deepcopy(self.professores_ocupados)
        )

    def __repr__(self):
        return f"Estado(alocadas={len(self.alocacoes)} disciplinas)"

    def __lt__(self, other):
        # Necessário para ordenação em heapq
        return id(self) < id(other)


print("Classes de estrutura de dados definidas!")

## 1. Modelagem Formal do Problema

In [ ]:
import heapq
import time
import pandas as pd
import numpy as np
from copy import deepcopy
from collections import defaultdict

# Configurar pandas para melhor visualização
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
print("Bibliotecas importadas com sucesso!")

# Alocação de Salas para Disciplinas (Timetabling Simplificado)
## Tema 6 - Trabalho de Inteligência Artificial UFPB

**Objetivo:** Resolver o problema de alocação de disciplinas em salas e horários usando dois algoritmos de busca:
- **A*** (com informação / heurística admissível)
- **IDDFS** (sem informação / híbrido BFS/DFS)

### Especificações do Problema

**Modelagem:**
- Estado: dicionário `{disciplina: (sala, horário)}`
- Ações: escolher disciplina não alocada e atribuir (sala, horário)
- Teste de objetivo: todas as disciplinas alocadas sem violações
- Custo: soma do desperdício de capacidade = Σ(capacidade_sala - alunos_disciplina)

**Restrições:**
1. Capacidade da sala ≥ número de alunos da disciplina
2. Mesmo professor não pode dar duas aulas no mesmo horário

**Heurística (A*):**
- h(n) = soma do menor desperdício possível para cada disciplina não alocada
- Admissível por construção (não sobreestima)

---